In [1]:
# convert coverage tracks in bedgraph (bdg) format of eg RNA-seq or ATAC-seq into my circos format

In [1]:
import pandas as pd
import pybedtools
import os

In [2]:
def generate_circos_coverage(bdg_file,karyotype_file):
    cov = pybedtools.BedTool(bdg_file)
    df = pd.DataFrame()
    with open(karyotype_file,'r') as f:
        for line in f.readlines():
            line = line.split(' ')
            loc = line[3]+' '+line[4]+' '+line[5]
            kary = pybedtools.BedTool(loc, from_string=True)
            res = cov.intersect(kary)
            del(kary)
            try:
                res = res.to_dataframe(disable_auto_names=True,header=None)
            except pd.errors.EmptyDataError:
                continue
            res["circos_chr"] = line[2]
            df = df.append(res)
    del cov
    df.columns = ['original_chr','start','end','score','circos_chr']
    df = df[["circos_chr",'start','end','score']]
    outfile = os.path.join(os.path.dirname(karyotype_file),os.path.basename(bdg_file)+'.circos.bdg')
    df.to_csv(outfile,sep='\t',header=False,index=False)
    return df
    
def import_bdg_file(bdg_file):
    df = pd.read_table(bdg_file,header=None,sep='\t',
                       names=['chr','start','end','score'])
    return df
def import_karyotype_file(karyotype_file):
    df = pd.read_table(karyotype_file,header=None,sep=' ',
                       names=['chr','-','n','chrom','start','end','color'],
                       usecols=['n','chrom','start','end'])
    return df

In [28]:
karyotype =  '/mnt/c/Users/ochapman/Documents/circos/D458/D458_ecDNA.karyotype'
import_karyotype_file(karyotype)
bdg = '/mnt/c/Users/ochapman/Documents/circos/case11/case11_rna.bdg'
import_bdg_file(bdg)
df = generate_circos_coverage(bdg,karyotype)
df


***** WARNING: File /mnt/c/Users/ochapman/Documents/circos/case11/case11_rna.bdg has inconsistent naming convention for record:
GL000008.2	0	177000	0

***** WARNING: File /mnt/c/Users/ochapman/Documents/circos/case11/case11_rna.bdg has inconsistent naming convention for record:
GL000008.2	0	177000	0

***** WARNING: File /mnt/c/Users/ochapman/Documents/circos/case11/case11_rna.bdg has inconsistent naming convention for record:
GL000008.2	0	177000	0

***** WARNING: File /mnt/c/Users/ochapman/Documents/circos/case11/case11_rna.bdg has inconsistent naming convention for record:
GL000008.2	0	177000	0

***** WARNING: File /mnt/c/Users/ochapman/Documents/circos/case11/case11_rna.bdg has inconsistent naming convention for record:
GL000008.2	0	177000	0

***** WARNING: File /mnt/c/Users/ochapman/Documents/circos/case11/case11_rna.bdg has inconsistent naming convention for record:
GL000008.2	0	177000	0

***** WARNING: File /mnt/c/Users/ochapman/Documents/circos/case11/case11_rna.bdg has inconsist

,circos_chr,start,end,score
0,1,1021689,1022000,41
1,1,1022000,1023000,7
2,1,1023000,1024000,2
3,1,1024000,1025000,1
4,1,1025000,1026000,0
...,...,...,...,...
61,22,47977000,47978000,602
62,22,47978000,47979000,526
63,22,47979000,47980000,0
64,22,47980000,47981000,668
